In [1]:
import pandas as pd
import jieba

In [2]:
dz_df = pd.read_csv(u'GBT14396-2016.csv')
dz_df.head()

,SJBXX01,SJBXX02,PYM,SZM,WBM
0,A00.000,霍乱，由于O1群霍乱弧菌，霍乱生物型所致,HLYYO1QHLHJHLSWXSZ,1321O1513511333131,FTMGO1VFTXAFTTTGRG
1,A00.100,霍乱，由于O1群霍乱弧菌，埃尔托生物型所致,HLYYO1QHLHJAETSWXSZ,1321O15135113133131,FTMGO1VFTXAFQRTTGRG
2,A00.900,霍乱,HL,13,FT
3,A01.000,伤寒,SH,34,WP
4,A01.001+K77.0*,伤寒性肝炎,SHXGY,34234,WPNEO


In [3]:
dz_df['class'] = [i[:3] for i in dz_df['SJBXX01']]
dz_df.head()

,SJBXX01,SJBXX02,PYM,SZM,WBM,class
0,A00.000,霍乱，由于O1群霍乱弧菌，霍乱生物型所致,HLYYO1QHLHJHLSWXSZ,1321O1513511333131,FTMGO1VFTXAFTTTGRG,A00
1,A00.100,霍乱，由于O1群霍乱弧菌，埃尔托生物型所致,HLYYO1QHLHJAETSWXSZ,1321O15135113133131,FTMGO1VFTXAFQRTTGRG,A00
2,A00.900,霍乱,HL,13,FT,A00
3,A01.000,伤寒,SH,34,WP,A01
4,A01.001+K77.0*,伤寒性肝炎,SHXGY,34234,WPNEO,A01


In [4]:
dzclass_df = pd.DataFrame(dz_df[['class', 'SJBXX02']].values,columns=['class','name'])
dzclass_df.head()

,class,name
0,A00,霍乱，由于O1群霍乱弧菌，霍乱生物型所致
1,A00,霍乱，由于O1群霍乱弧菌，埃尔托生物型所致
2,A00,霍乱
3,A01,伤寒
4,A01,伤寒性肝炎


In [5]:
g = dzclass_df.groupby('class')
nitems_eachclass = g.size().values
# nitems_eachclass
# print(g.size())

In [6]:
classset = set(dzclass_df['class'].values)
print('疾病共 % d 类' % len(classset))

疾病共  2047 类


In [7]:
classlist = sorted(list(classset))
# print(classlist)

In [8]:
# strlist = []
# for index in classlist:
#     localstrlist = dzclass_df['name'][dzclass_df['class']==index].values
#     tempstr = ''
#     for s in localstrlist:
#         tempstr += s
#     strlist.append(tempstr)
# print(strlist)
# print(len(strlist))

In [9]:
print(len(classlist))
dzclass_df = dzclass_df.sort_values(by=['class'])  
# 因为之前对class 进行了排序,得到的nitems_eachclass是class排序后的对应结果, 所以这里要对dzclass_df重新排序
# dzclass_df

2047


In [10]:
'''较为快速的实现, 比上个版本快5倍'''
strlist = []
start_index = 0
for index in nitems_eachclass:
    localstrlist = dzclass_df['name'][start_index:start_index+index].values
    tempstr = ''
#     print(localstrlist)
    for s in localstrlist:
        tempstr += ' ' + s
    strlist.append(tempstr.strip())
    start_index += index
# print(strlist)
print(len(strlist))

2047


In [16]:
strlist[0]

'霍乱，由于O1群霍乱弧菌，霍乱生物型所致 霍乱，由于O1群霍乱弧菌，埃尔托生物型所致 霍乱'

In [20]:
from collections import Counter
seg_list = jieba.cut(strlist[1], cut_all=True)  # cut_all=True表示全模式。默认为精确模式
c = Counter()
for x in seg_list:
#     print(x)
    if len(x)>1 and x != '\r\n':  # 除去单字符以及换行(\r把光标移动到行首)
        c[x] += 1
print(c)

Counter({'伤寒': 8, '副伤寒': 4, '败血': 1, '膜炎': 1, '脑膜炎': 1, '性肝炎': 1, '血症': 1, '伤寒杆菌': 1, '败血症': 1, '杆菌': 1, '脑膜': 1, '肝炎': 1})


In [72]:
jieba.load_userdict("disease_dict.txt")  # load my dict.txt

In [73]:
def wordscounter(strlist):
    dzlist = []
    for s in strlist:
        c = Counter()
        seg_list = jieba.cut(s, HMM=True)
        for x in seg_list:
            if len(x)>1 and x != '\r\n':  # 除去单字符以及换行(\r把光标移动到行首)
                c[x] += 1
#         print(c.most_common(1))
        if c.most_common(1):
            dzlist.append(c.most_common(1)[0][0])
    return dzlist

In [77]:
dzlist = wordscounter(strlist)
dzset = set(dzlist)
print(len(dzlist), len(dzset))
print(dzset)

2046 1207
{'短暂性关节炎', '新生儿破伤风', '催眠', '孕产妇', '沙门', '虫病', '前臂损伤', '有害效应', '家具', '高铁血红蛋白血症', '过多', '虐待', '乳突胆脂瘤', '酒渣鼻', '需要', '菌血症', '骨折', '营养缺乏', '肌腱端病', '畸形', '鼠咬热', '阴茎体恶性肿瘤', '调节', '过敏', '口腔', '色素', '皮肤良性肿瘤', '黄疸', '碰撞', '自害', '梅毒', '角膜', '结缔组织和软组织恶性肿瘤', '绦虫病', '低血糖', '内分泌', '老年性痴呆', '宫颈上皮内肿瘤', '多胎妊娠', '甲状腺功能减退', '有机', '人格障碍', '子宫', '人格', '鼻炎', '巨细胞', '埋伏牙', '萎缩卵', '房间隔缺损', '腹部', '主动脉瓣', '设施', '接触毒蜘蛛', '幼年', '急躁和愤怒', '脐带', '恶性肿瘤家族史', '球蛋白', '个人', '剥脱性皮炎', '视神经炎', '库欣综合征', '慢性外阴炎', '白喉', '振动', '脑室', '病毒性出血热', '登革热', '醛固酮', '立克次体病', '角化病', '肺动脉瓣', '坏疽', '气管恶性肿瘤', '巩膜黑变病', '静脉', '出血', '其他哺乳动物咬伤', '受害者', '滑脱', '精索扭转', '改善', '脑卒中', '关节软骨恶性肿瘤', '甲状旁腺功能亢进', '管子', '肺荚膜组织胞浆菌病', '钩端螺旋体病', '先天性碘缺乏综合征', '皮肤', '暴力', '吉兰', '急性', '下肢', '佝偻病', '并指', '支气管哮喘', '衣原体', '扁桃体柱恶性肿瘤', '外因', '视觉障碍', '用腐蚀性物质进行加害', '抑郁发作', '特指', '胆囊恶性肿瘤', '呼吸停止', '语音障碍', '城市狂犬病', '肾炎综合征', '分泌', '产褥期', '贫困', '为抗', '硬红斑', '皮肌炎', '不规则', '乳头', '其他人', '曲霉', '公共汽车', '链球菌作为分类于其他章疾病的原因', '酸性', '特殊', '遗尿', '泌尿科', '陈旧性', '童年情绪障碍', '全身性硬

In [75]:
wordscounter(strlist[0])


[]